
# 📘 Ingestion de fichiers csv avec Auto Loader

# 🔹 1. Définir les chemins de stockage

In [0]:
 # Récupération de la clé de stockage depuis le secret scope "key3"
# Remplace 'blobstorage_scope' par le nom réel de ton secret dans ce scope
storage_account_key = dbutils.secrets.get(scope="key_adls", key="storage_key")
# Configuration de l'accès au compte de stockage Azure
spark.conf.set("fs.azure.account.key.comptestorage0000000001.dfs.core.windows.net", storage_account_key)

In [0]:
# Chemin du dossier contenant le fichier .csv (sans inclure le nom du fichier)
source_path = "abfss://test@comptestorage0000000001.dfs.core.windows.net/input/"

# Emplacement pour stocker les checkpoints Auto Loader
checkpoint_path = "abfss://test@comptestorage0000000001.dfs.core.windows.net/_checkpoint/autoloader_csv"

# Nom de la table Delta de destination
target_table = "hive_metastore.dev_database.ma_table_csv"

###  1. Lire un fichier statique pour inférer le schéma et créer la table (avec .read, pas .readStream)

In [0]:
df_sample = (spark.read
    .format("csv")
    .option("header", "true")
    .option("sep", ";")
    .option("inferSchema", "true")
    .load(source_path))  # fichier ou dossier déjà existant

# Nettoyage des noms de colonnes
df_sample = df_sample.toDF(*[col.replace(";", "_").replace(" ", "_") for col in df_sample.columns])

In [0]:
display(df_sample)

In [0]:
df_sample = (spark.read
    .format("csv")
    .option("header", "true")
    .option("sep", ";")
    .option("inferSchema", "true")
    .load(source_path))

# Nettoyage des noms de colonnes
df_sample = df_sample.toDF(*[col.replace(";", "_").replace(" ", "_") for col in df_sample.columns])

# Enregistrement propre
df_sample.write.format("delta").mode("overwrite").saveAsTable("hive_metastore.dev_database.ma_table_csv")

##  🔹 2. Lecture en streaming avec Auto Loader

In [0]:
 
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "csv")
      .option("cloudFiles.schemaLocation", checkpoint_path)
      .option("header", "true")           # Le fichier CSV a-t-il un en-tête ?
      .option("sep",";")
      .option("inferSchema", "true")      # Auto-détection du schéma
      .option("cloudFiles.schemaEvolutionMode", "rescue")
      .load(source_path))



df= df.toDF(*[col.replace(";", "_").replace(" ", "_") for col in df.columns])

In [0]:
display(df)

## 🔹 3. Écriture vers une table Delta (Unity Catalog ou Live metastore)

## 3.1 Vérifiez d'abord les tables catalogues

In [0]:
%sql
SHOW CATALOGS;



## 3.2 Spécifier le catalogue a utiliser: Exemple avec hive_metastore

In [0]:
%sql
USE CATALOG hive_metastore;

## 3.3 Créer un schéma dans un catalogue existant (si besoin)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS hive_metastore.dev_database;

In [0]:
query =(df.writeStream
    .format("delta")    
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")   # permettre que le schéma de la Delta table évolue (par exemple ajout de colonnes) 
    .outputMode("append") 
    .start(target_table))      # écriture dans Unity Catalog


display(query)

In [0]:
display(query)

## 4. Vérification des données dans la table

In [0]:
%sql
SELECT * FROM hive_metastore.dev_database.ma_table_csv;

In [0]:
query.status
query.lastProgress